In [10]:
# File for data cleaning
import pandas as pd

In [11]:
full_df = pd.read_csv("Wimbledon_featured_matches.csv")

In [12]:
test_df = full_df.loc[full_df["match_id"] == "2023-wimbledon-1301"]

In [13]:
test_df.head(30)

,match_id,player1,player2,elapsed_time,set_no,game_no,point_no,p1_sets,p2_sets,p1_games,...,p2_break_pt_won,p1_break_pt_missed,p2_break_pt_missed,p1_distance_run,p2_distance_run,rally_count,speed_mph,serve_width,serve_depth,return_depth
0,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:00:00,1,1,1,0,0,0,...,0,0,0,6.000,7.840,2,95.0,BC,NCTL,ND
1,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:00:38,1,1,2,0,0,0,...,0,0,0,5.253,7.094,1,118.0,B,CTL,ND
2,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:01:01,1,1,3,0,0,0,...,0,0,0,13.800,19.808,4,120.0,B,NCTL,D
3,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:01:31,1,1,4,0,0,0,...,0,0,0,51.108,75.631,13,130.0,BW,CTL,D
4,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:02:21,1,1,5,0,0,0,...,0,0,0,0.649,0.813,1,112.0,W,NCTL,NaN
5,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:02:50,1,1,6,0,0,0,...,0,0,0,5.291,4.249,2,97.0,BW,NCTL,ND
6,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:03:33,1,1,7,0,0,0,...,0,0,0,6.817,17.821,1,109.0,W,CTL,D
7,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:04:01,1,1,8,0,0,0,...,0,0,0,17.503,18.067,6,105.0,B,NCTL,ND
8,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:04:48,1,1,9,0,0,0,...,0,0,0,34.489,24.594,7,128.0,BC,CTL,D
9,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:05:32,1,1,10,0,0,0,...,0,0,0,21.235,27.427,5,110.0,B,NCTL,ND


In [25]:
ivan_list = ["_var1", "_var2", "_var3"]
def f(i): return [f"p{i}" + string for string in ivan_list]


f(1)

['p1_var1', 'p1_var2', 'p1_var3']

In [14]:
# score_point_diff = Player_i_points - Player_notI_points
# serve_1_i_broken = server == i and p_i_Ace =! 1
# P_i_BrealPointToT = BreakPoint - BreakPoint miss


def get_row_momentum(row, i):
    momentum = 0
    not_i = 2 if i == 1 else 1
    score_point_diff = row[f"p{i}_points_won"] - row[f"p{not_i}_points_won"]  # new row
    # TODO: new row
    serve_1_broken = 1 if row["server"] == not_i and row[f"p{not_i}_ace"] != 1 else 0

    # TODO: unforced error

    # TODO: add row momentum


# TODO: add in double faults


def get_set_momentum(df, set_num, i):
    df_set = df.loc[::, df["set"] == set_num]  # FIXXXXXXXXX
    for row in df_set:
        # TODO: change to create new or alter existing df
        # TODO: Winning long rallys
        # Create col for set and game momentum
        get_row_momentum(row, i)


# TODO: create game momentum

# TODO: Do something useful w it

List of events:
- being down/up +- 2 (Game and Set)
- Breaking a first serve/ Ace
- Winning long rally
- Pregame odds(maybe) + Lucky points
- Making/Not making break points
- double fault
- unforced error

In [15]:
test_row = test_df.iloc[11, :]

rally_df = test_df.loc[test_df["rally_count"] > 9]


# rally_df = rally_df.loc[rally_df["server"] == 2]
# rally_df = rally_df.loc[rally_df["serve_no"] == 1]

In [16]:
rally_df[["rally_count", "server"]]  # >9 long rally

InvalidIndexError: (['rally_count', 'server'], slice(None, 3, None))

In [22]:
i = 1
not_i = 2

# score_point_diff = test_row[f"p{i}_points_won"] - \
#     test_row[f"p{not_i}_points_won"]  # works

# serve_1_broken = 1 if test_row["server"] == not_i and test_row["rally_count"] > 1 else 0
# WORKS

test_row = test_df.iloc[1:3, :]
test_row["match_id"]

1    2023-wimbledon-1301
2    2023-wimbledon-1301
Name: match_id, dtype: object

In [ ]:
import numpy as np